In [1]:
import requests
from bs4 import BeautifulSoup
import time 
import random 
import re
from fake_useragent import UserAgent
import datetime
from elasticsearch import Elasticsearch
from tqdm import tqdm

In [2]:
class KanobuArticle:
    def __init__(self):
        self.date=""
        self.time=""
        self.words=""
        self.rubr=""
        self.hashtag=""
        self.author=""
        self.head=""
        self.text=""
        
    # Конвертация в JSON.
    def toJSON(self):
        res='{"date":"'+self.date+'", "time":"'+self.time+'", "words":"'+self.words+'", "rubrics":"'+self.rubr+'", "hashtag":"'+self.hashtag+'", "title":"'
        +self.head+'", "author":"'+self.author+'","text":"'
        res+=self.text.replace('"', '\\"')+'"}'
        return res

    # Конвертация в словарь.
    def toDict(self):
        res={"date":self.date, "time":self.time, "rubrics":self.rubr, "hashtag":self.hashtag, \
             "title":self.head, "words":self.words, "author":self.author,"text":self.text.replace('"', '\\"')}
        return res

In [3]:
def GetPagesArticles(number_of_pages):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    url = 'https://kanobu.ru/news/?page='
    links = []
    for page in range(1, number_of_pages+1):
        r = requests.get(url + str(page), headers = headers)
        time.sleep(random.randint(1,6))
        content = r.content
        html = content.decode('UTF-8')
        soup = BeautifulSoup(html)
        
        for link in soup.find_all('a', attrs={'class':'c-item_foot d-b t-l'}):
            links.append(link.get('href'))
    
    return links    

In [4]:
def clean_text (text):
    text = text.replace('\xa0', ' ')
    text = text.replace ('\n', ' ')
    text = text.replace ('  ', ' ')
    return text

In [5]:
def GetArticles(links):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    l = 'https://kanobu.ru'
    articles = []
    for link in links:
        art = KanobuArticle()
        page = requests.get(l+link, headers = headers)
        time.sleep(random.randint(1,3))
        content2 = page.content
        html2 = content2.decode('UTF-8')
        soup2 = BeautifulSoup(html2)
        
        date = soup2.find('div', attrs={'class':'c-title_footer'}).get_text()
        art.date = date.split(', ')[0]
        art.time = date.split(', ')[1]
        
        author = soup2.find('a', attrs={'class':'c-title_author'}).get_text()
        art.author = author
        
        text = soup2.find('div', attrs={'class':'c-detail_content'}).get_text()
        art.text = clean_text(text)
        
        words = len(clean_text(text).split())
        art.words = words
        
        title = soup2.find('h1', attrs={'class':'c-title_body'}).get_text()
        art.head = clean_text(title)
        
        rubric = soup2.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
        r = rubric.split(' | ')
        art.rubr = r[0]
        art.hashtag = r[1]
        #print(art)
        #print(text)
        #print(date)
        #print(author)
        print(title)
        print(words)
        #print(r[0])
        #print(r[1])
        #print(rubric)
        articles += [art]
    return articles

In [6]:
articles = GetArticles(GetPagesArticles(1))

Режиссер фильма «Поезд в Пусан 2» поделился подробностями сюжета
199
Патрик Стюарт читает Шекспира, чтобы пережить карантин (видео)
193
Меган Маркл может озвучить персонажа в «Симпсонах»
183
«Я полностью выздоровела»: Ольга Куриленко заявила, что вылечилась от коронавируса
162
В России снимут сериал о киберспорте
156
Kojima Productions добавляет фоторежим в Death Stranding на PS4. А сам гений играет в Half-Life Alyx
195
Qiwi объявила о закрытии онлайн-банка «Рокетбанк»
138
Microsoft рассказала о серьезной уязвимости всех Windows, но проблему решат только в апреле
157
Никакой UNO от Little Big: участники Евровидения должны представить новые песни
181
«Ковидиот» — новое слово в Urban Dictionary. У него несколько значений
168
В сети появился новый скриншот Star Wars 1313 — отмененной игры по «Звездным войнам»
140
Фанаты Marvel разнесли в пух и прах новых супергероев
284
BioWare выпустит книгу в честь 25-летия. Ее посвятят истории студии
156
Представлен Redmi Max 98: дорогой гигантский 4К-

### База

In [7]:
es=Elasticsearch()

In [8]:
es

<Elasticsearch([{}])>

In [9]:
es.indices.delete(index="kanobu") 

{'acknowledged': True}

In [10]:
es.indices.create(index="kanobu")

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'kanobu'}

In [11]:
mapit={"article":{"properties":{"author":{"type":"text"},
                                "date":{"type":"text"},
                                "time":{"type":"date", "format":" HH:mm"},
                                "words":{"type":"double"},
                                "hashtag":{"type":"text","analyzer" : "russian"},
                                "rubrics":{"type":"text","analyzer" : "russian"},
                                "text":{"type":"text","analyzer" : "russian"},
                                "title":{"type":"text","analyzer" : "russian"}}}}

In [12]:
es.indices.put_mapping(index="kanobu", doc_type='article', body=mapit, include_type_name=True)

{'acknowledged': True}

In [13]:
es.indices.get_alias("*")

{'kanobu': {'aliases': {}}}

In [14]:
%%time
i = 0
for article in tqdm(articles):
    es.index(index="kanobu", doc_type='article', id = i, body=article.toDict())
    i+=1

100%|█████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 110.05it/s]

Wall time: 174 ms


In [15]:
res = es.search(index="kanobu", body={"query": {"match_all": {}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 1.0,
  '_source': {'date': '24 марта 2020',
   'time': '16:30',
   'rubrics': 'Кино и сериалы',
   'hashtag': '#Поезд в Пусан 2',
   'title': 'Режиссер фильма «Поезд в Пусан 2» поделился подробностями сюжета',
   'words': 199,
   'author': 'Анастасия Яблоновская',
   'text': 'В интервью изданию Screen Daily режиссер и сценарист Ен Сан-хо раскрыл детали сюжета предстоящего фильма «Поезд в Пусан 2: Полуостров». По его словам, новая лента не будет прямым продолжением нашумевшего зомби-хоррора «Поезд в Пусан» 2016 года. Ен Сан-хо рассказал, что действие нового фильма развернется в той же вселенной спустя четыре года после событий оригинальной ленты. Однако история и герои будут совсем другими. «Из-за появления зомби государственная власть в Корее уничтожена. Ничего не осталось, только географические особенности места. Поэтому фильм называется „Полуостров“», — рассказал постановщик. Главным героем станет бывший солдат, ко

### Добавление в базу новой статьи по ссылке

In [16]:
def add_article(link):
    articles = []
    art = KanobuArticle()
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2225.0 Safari/537.36'}
    page = requests.get(link, headers = headers)
    content = page.content
    html = content.decode('UTF-8')
    soup = BeautifulSoup(html)
    
    date = soup.find('div', attrs={'class':'c-title_footer'}).get_text()
    art.date = date.split(', ')[0]
    art.time = date.split(', ')[1]
        
    author = soup.find('a', attrs={'class':'c-title_author'}).get_text()
    art.author = author
        
    text = soup.find('div', attrs={'class':'c-detail_content'}).get_text()
    art.text = clean_text(text)
        
    words = len(clean_text(text).split())
    art.words = words
        
    title = soup.find('h1', attrs={'class':'c-title_body'}).get_text()
    art.head = clean_text(title)
        
    rubric = soup.find('div', attrs={'class':'c-title_head t-t-u'}).get_text()
    r = rubric.split(' | ')
    art.rubr = r[0]
    art.hashtag = r[1]
    
    #print(art)
    #print(text)
    #print(date)
    #print(author)
    #print(title)
    #print(words)
    #print(r[0])
    #print(r[1])
    
    articles += [art]
    
    res = es.search(index="kanobu", body={"query": {"match_all": {}}})
    maxid = res['hits']['total']['value']
    
    for article in articles:
        es.index(index="kanobu", doc_type='article', id = maxid, body=article.toDict())
        
    print('Done')

In [17]:
link = 'https://kanobu.ru/news/entoni-hopkins-syigral-nafortepiano-dlya-svoego-kota-i-pokoril-sotsseti--422323/'
add_article(link)

Done


### Мэтч по id новой статьи

In [18]:
es.get(index="kanobu", doc_type='article', id=18)

{'_index': 'kanobu',
 '_type': 'article',
 '_id': '18',
 '_version': 1,
 '_seq_no': 18,
 '_primary_term': 1,
 'found': True,
 '_source': {'date': '20 марта 2020',
  'time': '12:59',
  'rubrics': 'Кино и сериалы',
  'hashtag': '#энтони хопкинс',
  'title': 'Энтони Хопкинс сыграл на фортепиано для своего кота и покорил соцсети ',
  'words': 164,
  'author': 'Анастасия Яблоновская',
  'text': 'Умиротворяющим видео поделился с подписчиками Энтони Хопкинс. Он сыграл на фортепиано для своего кота. Судя по ролику, питомец музыкальным талантом актера доволен. «Niblo заботится о том, чтобы я оставался здоровым, а взамен требует себя развлекать», — отметил Хопкинс в подписи к видео. Баста дал концерт в онлайне. Все, чтобы спасти нас от скуки Ролик получил большую популярность в сети. На Facebook у него уже более 6 млн. просмотров. В комментариях поклонники актера назвали видео замечательным и очень милым. А еще пожелали кумиру беречь здоровье. Вчера вдохновляющим роликом поделилась с фанатами ак

## Удаление статьи по id

In [19]:
es.delete(index="kanobu",doc_type="article",id=18)

{'_index': 'kanobu',
 '_type': 'article',
 '_id': '18',
 '_version': 2,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 19,
 '_primary_term': 1}

### Мэтч по заголовку

In [20]:
res = es.search(index="kanobu", body={"query": {"match": {"title": "россия"}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '4',
  '_score': 3.2563362,
  '_source': {'date': '24 марта 2020',
   'time': '15:24',
   'rubrics': 'Киберспорт',
   'hashtag': '#россия',
   'title': 'В России снимут сериал о киберспорте',
   'words': 156,
   'author': 'Сергей Сурепин',
   'text': 'Площадка «КиноПоиск HD» подготовила новый эксклюзив. Сервис выпустит сериал о жизни молодого киберспортсмена — шоу получило название «Хэдшот». «Кинопоиск» открыл бесплатный доступ к онлайн-кинотеатру Главный герой сериала — старшеклассник Денис. Как сообщили в пресс-службе «КиноПоиска», молодой человек каждый день терпит издевательства в школе, однако на помощь ему приходит его собственный аватар из компьютерной игры — lllRAM. Благодаря советам виртуального персонажа парень станет более уверенным в себе. После этого он попадет в мире профессионального киберспорта, где его ожидают первые деньги, новые знакомства, а также шанс добиться внимания любимой девушки. Жизнь Ельцина станет сериал

### Мэтч по дате

In [21]:
res = es.search(index="kanobu", body={"query": {"match": {"date": "22 марта 2020"}}})
res["hits"]["hits"]

[{'_index': 'kanobu',
  '_type': 'article',
  '_id': '0',
  '_score': 0.050635617,
  '_source': {'date': '24 марта 2020',
   'time': '16:30',
   'rubrics': 'Кино и сериалы',
   'hashtag': '#Поезд в Пусан 2',
   'title': 'Режиссер фильма «Поезд в Пусан 2» поделился подробностями сюжета',
   'words': 199,
   'author': 'Анастасия Яблоновская',
   'text': 'В интервью изданию Screen Daily режиссер и сценарист Ен Сан-хо раскрыл детали сюжета предстоящего фильма «Поезд в Пусан 2: Полуостров». По его словам, новая лента не будет прямым продолжением нашумевшего зомби-хоррора «Поезд в Пусан» 2016 года. Ен Сан-хо рассказал, что действие нового фильма развернется в той же вселенной спустя четыре года после событий оригинальной ленты. Однако история и герои будут совсем другими. «Из-за появления зомби государственная власть в Корее уничтожена. Ничего не осталось, только географические особенности места. Поэтому фильм называется „Полуостров“», — рассказал постановщик. Главным героем станет бывший со

### Виджеты

In [22]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from ipywidgets import interact, interactive, fixed, interact_manual

"Требования к нереляционным БД
2 - нереляционная БД
2 - красивая структура БД
2 - интерфейс позволяет класть, доставать, удалять данные (проводить операции CRUD - Create, Read, Update, Delete)
2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...)
2 - зависит от БД: Redis - использование ключей, хешей, ...; Neo4j - нахождение путей не только к соседним вершинам, операции с графами; MongoDB, ElasticSearch - работа с текстами или географией; остальные БД - будем договариваться."

### 1 показ всех статей

In [23]:
button1 = widgets.Button(description='Match all articles') 
out = widgets.Output()

def on_button_clicked(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"query": {"match_all": {}}})
        for part in res["hits"]["hits"]:
            print(part + '\n')

In [25]:
button2 = widgets.Button(description='Match with options')
out = widgets.Output()

def on_button_clicked2(_):
    with out:
        clear_output()
        res = es.search(index="kanobu", body={"query": {"match": {menu.value : text.value}}})
        for part in res["hits"]["hits"]:
            print(part)

In [27]:
button1.on_click(on_button_clicked)
button2.on_click(on_button_clicked2)

In [28]:
text1 = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [29]:
menu = widgets.Dropdown(
       options=['id', 'title', 'date', 'author', 'hashtag', 'rubrics'],
       value='id',
       description='How to find?')

In [30]:
text = widgets.Text(
    value='',
    placeholder='Type what you want to find',
    description='Text:',
    disabled=False
)

In [32]:
box = widgets.VBox([button1, menu, text, button2, out])
box

- поиск по Id выполняется по другому (придумать как добавить его в ту же кнопку)
- кнопка для добавления статьи по ссылке
- не хватает операции update (например, "положи по существующему индексу другую статью"
- "2 - два действия помимо CRUD (сортировка, группировка, агрегация, ...) "